In [1]:
import chardet
from os import listdir
from os.path import isfile, splitext
import pandas as pd
import numpy as np
import string


In [4]:
print('Comenzando procesamiento')
# Globales
ruta_data = 'datasets/'
ruta_csv = 'csv/'
archs = listdir(ruta_data)

# Obtener la ruta de la carpeta que contiene los archivos CSV
ac1='ratings/1'
ac2='ratings/2'
ac3='ratings/3'
ac4='ratings/4'
ac5='ratings/5'
ac6='ratings/6'
ac7='ratings/7'
archivos = [ac1,ac2,ac3,ac4,ac5,ac6,ac7]
lista_df = []
for archivo in archivos:
            df1 = pd.read_csv(archivo+".csv")
            lista_df.append(df1)

df_rating = pd.concat(lista_df)
df_rating = df_rating.rename(columns={'movieId': 'id'})
df_rating = df_rating.rename(columns={'rating': 'score'})

# Detectamos y seleccionamos los archivos a procesar
extensiones = ['.csv']
arch_fallos = []
arch_ok = []
# Recorremos los archivo encontrados    
for arch in archs:
        if isfile(ruta_data + arch):
                # Separamos en nombre y ext de arhivo
                nombre_arch, ext_arch = splitext(arch)
                # Los clasificamos
                if ext_arch in extensiones:
                    arch_ok.append(arch)
                else:
                    arch_fallos.append(arch)   

        # Procesamos los archivos CSV
    
for arch in arch_ok:
        nombre_arch, ext_arch = splitext(arch)
        if ext_arch == '.csv':
                with open(ruta_data + arch, 'rb') as f:
                    # Detectamos el encoding
                    resultado = chardet.detect(f.read())
                    encod = resultado['encoding']
                df_csv = pd.read_csv(ruta_data +  arch, encoding=encod)
                df_rat=df_rating.copy()

                # Crear columna ID
                
                df_csv['id'] = nombre_arch[0] + df_csv['show_id'].astype(str)
                df_csv.insert(0, "id", df_csv.pop("id"))

                # reemplazamos valores nulos de la columna rating por G
                df_csv['rating'] = df_csv['rating'].fillna('G')

                # cambiamos el formato de date_added
                df_csv['date_added'] = pd.to_datetime(df_csv['date_added'], format='%B %d, %Y', errors='coerce')
                df_csv['date_added'] = df_csv['date_added'].dt.strftime('%Y-%m-%d')

                # convertimos las mayusculas en minusculas
                df_csv = df_csv.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

                # separar la columna duration en duration_int y duration_type
                df_csv["duration_int"] = df_csv["duration"].str.extract('(\d+)', expand=False)
                df_csv["duration_int"] = pd.to_numeric(df_csv["duration_int"], errors='coerce')
                df_csv["duration_type"] = df_csv["duration"].astype(str)
                df_csv["duration_type"] = df_csv["duration_type"].str.translate(str.maketrans('', '', string.digits))
                df_csv["duration_type"] = df_csv["duration_type"].str.replace("seasons", "season")

                # Agregar la columna score
                grupo = df_rat.groupby('id')
                resultado = round(grupo['score'].mean(),2)
                resultado = resultado.to_frame().reset_index()
                dfconbinado = pd.merge(df_csv,resultado,on='id',how='left')
                        
                dfconbinado.to_csv(ruta_csv + nombre_arch + '.csv', index=False, encoding='utf8')        
        
print('Fin del Procesamiento')

Comenzando procesamiento
Fin del Procesamiento
